<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Modificado en 2019-1 al 2025-2 por Equipo Docente IIC2233. </font>
</p>

# Tabla de contenidos

1. [*Networking* con Python](#Networking-con-Python)
    1. [`Sockets`](#Sockets)
        1. [Comandos básicos de sockets](#comandos-básicos-de-sockets)
            1. [Cliente TCP](#Cliente-TCP)
            2. [Servidor TCP](#Servidor-TCP)
            3. [Ejemplo base](#ejemplo-base)

# *Networking* con Python

En este material, veremos cómo implementar programas de **bajo nivel** en Python para que puedan comunicarse con otro programa, el cual puede encontrarse en el mismo computador o en otro.

## *Sockets*

Un ***socket*** es un objeto del sistema operativo que permite a un programa transmitir y recibir datos desde y hacia otro programa corriendo en otra máquina, o en la misma máquina pero en otro puerto. Son nuestro "punto de entrada y salida a/desde la red" en nuestro código. Para obtener un *socket*, necesitamos (1) especificar qué tipo dirección IP usaremos (IPv4 o IPv6), y qué protocolo de transporte usaremos (TCP ó UDP).

En Python, los *sockets* son provistos por el módulo `socket`. Para crear un *socket*, debemos crear una instancia de la clase `socket(family, type)`, donde `family` es el tipo de dirección IP que usaremos, y `type` es el protocolo de transporte. Algunos de los valores permitidos son:

- `family`
    - `AF_INET` para direcciones IPv4
    - `AF_INET6` para direcciones IPv6
- `type`
    - `SOCK_STREAM` para TCP
    - `SOCK_DGRAM` para UDP

Por ejemplo, para crear un *socket* TCP con IPv4, lo hacemos de la siguiente manera.

In [1]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### Comandos básicos de sockets

Veamos cómo implementar una arquitectura cliente-servidor, utilizando _sockets_. Para poder implementarlo, utilizaremos varios conceptos del notebook "2-networking_conceptos_básicos" de la semana anterior, te recomendamos revisarlo si no recuerdas la materia.

Para poder comunicarnos utilizando _sockets_ será necesario seguir una serie de pasos definidos para establecer la comunicación entre el cliente y el servidor, y asegurar que se envíen y reciban correctamente los mensajes.

Los pasos que se deben seguir se explican en la siguiente imagen:

<img src="imgs/socket.jpg" alt="Flujo sockets entre cliente y servidor" style="height: 400px"/>
<p>  </p>

Fuente: [**HowDev:** Socket programming in Python](https://how.dev/answers/socket-programming-in-python)

> **NOTA:** Por simplicidad, en el siguiente notebook solo enseñaremos cómo implementar un comunicación utilizando el protocolo TCP. Si quieres aprender cómo se utiliza el protocolo UDP, revisa el notebook bonus de esta semana. 

#### Cliente TCP

Para establecer una conexión TCP necesitamos (1) la dirección IP del *host* destinatario, y (2) el puerto del *host* destinatario. Para conectarse a un servidor TCP usamos el método `connect((host, port))` del *socket*. El método recibe una tupla, cuyo primer elemento debe ser una dirección IP acorde a lo que especificamos al momento de crear el *socket*, y el segundo elemento es el puerto al que nos queremos conectar. 

Opcionalmente, el primer elemento de la tupla puede ser un *string* con el *hostname*, como por ejemplo, `python.org`, `github.com` o `canvas.uc.cl`. En ese caso, el sistema utilizará el protocolo DNS para obtener la  dirección IP correspondiente a ese *hostname*.

Por ejemplo, hagamos un cliente que se conecte a `104.18.32.7` en el puerto 80 vía TCP.

In [2]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('104.18.32.7', 80))

Si nos queremos conectar a [stackoverflow.com](https://stackoverflow.com) también lo podemos hacer sin conocer su dirección IP.

In [3]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('stackoverflow.com', 80))

Llegó la hora de enviar algún mensaje. Los *sockets* sólo transmiten *bytes*, por lo que cualquier cosa que enviemos debemos codificarla. Para enviar los *bytes* de un mensaje, se puede ocupar uno de los siguientes métodos de socket:

- `send(bytes)`: Trata de enviar el mensaje almacenado en la variable *bytes*. Sin embargo, puede que envie solo parte de este en vez del mensaje completo, o el envío resulte en error. Debido a lo anterior, este método retorna la cantidad de *bytes* que fue efectivamente enviada, dejando como responsabilidad al programador el llamar nuevamente `send` para enviar los *bytes* restantes hasta que se haya enviado el mensaje completo. Nos daremos cuenta de esto cuando la suma de la cantidad de bytes enviados en cada llamada al método sea igual al largo del mensaje original.
- `sendall(bytes)`: Este método se asegura de enviar todos los *bytes* del mensaje, haciendo uso por debajo del método `send` hasta que el mensaje se haya enviado en su totalidad. Sin embargo, en caso de haber un error en el envío, este método solo lanzará un error, por lo que no podremos saber cuántos bytes efectivamente se habían enviado de forma previa a que ocurriera error.

En el siguiente ejemplo, los *bytes* contienen un mensaje en el formato HTTP, y corresponde a una **solicitud HTTP** (*HTTP Request*) que puede ser comprendida por un servidor web. Los enviaremos mediante el método `sendall`:

In [4]:
solicitud = 'GET / HTTP/1.1\nHost: stackoverflow.com\n\n'
sock.sendall(solicitud.encode('ascii'))

¿Qué nos habrá respondido el servidor? Para saberlo, debemos ocupar el método `recv(buffer)`, que nos retornará un objeto `bytes`. El parámetro que recibe este método es la cantidad __máxima__* de *bytes* que será leída. La documentación sugiere colocar una potencia de 2 pequeña, como 4096.

*Es la cantidad máxima, porque lo que hace es sacar los bytes que hayan llegado al socket en ese instante. Si es que en el socket habían llegado 20 bytes al momento de llamar a `sock.recv(4096)`, entonces el método me entregará solo 20 bytes.

In [5]:
data = sock.recv(4096)
print(data.decode('ascii'))

HTTP/1.1 301 Moved Permanently
Date: Mon, 20 Oct 2025 17:53:19 GMT
Content-Type: text/html
Content-Length: 167
Connection: keep-alive
Cache-Control: max-age=3600
Expires: Mon, 20 Oct 2025 18:53:19 GMT
Location: https://stackoverflow.com/
Set-Cookie: __cf_bm=WMRXUg00zZbVb7qGcyGyr.e1srEMXfUay0us64uW5Ig-1760982799-1.0.1.1-SzHwT4pQkZtusD_C.xugMcXCO68mpEZ_XPS4XsPFoZq8zYen2ur3byq3KT2fpLfWIcshv0zh4AA7J_Z3ABH6dhIE1Lt2ef2N.BM4jTuUuBU; path=/; expires=Mon, 20-Oct-25 18:23:19 GMT; domain=.stackoverflow.com; HttpOnly
X-DNS-Prefetch-Control: off
Server: cloudflare
CF-RAY: 991a5fbdbd7fb440-SCL

<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>cloudflare</center>
</body>
</html>



El último paso, pero no el menos importante: **debemos cerrar la conexión**; de lo contrario, si nuestro programa continúa, estará ocupando recursos que no utiliza.

In [6]:
sock.close()

Hagamos el ejercicio completo en una sola celda. Además, aprovechemos de manejar las excepciones pues, en *networking*, es muy probable que se produzcan errores. La clase `ConnectionError` es la clase base para distintos errores relacionados con problemas de conexión, por lo que capturando su excepción podremos responder a cualquier problema que ocurra en este sentido.

In [7]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

host = "stackoverflow.com"
port = 80

try:
    sock.connect((host, port))
    sock.sendall('GET / HTTP/1.1\nHost: stackoverflow.com\n\n'.encode('utf-8'))
    data = sock.recv(4096)
    print(data.decode('utf-8'))
except ConnectionError as e:
    # ConnectionError es la clase base de BrokenPipeError, ConnectionAbortedError,
    # ConnectionRefusedError y ConnectionResetError
    print("Ocurrió un error.")
finally:
    # ¡No olvidemos cerrar la conexión!
    sock.close()

HTTP/1.1 301 Moved Permanently
Date: Mon, 20 Oct 2025 17:53:23 GMT
Content-Type: text/html
Content-Length: 167
Connection: keep-alive
Cache-Control: max-age=3600
Expires: Mon, 20 Oct 2025 18:53:23 GMT
Location: https://stackoverflow.com/
Set-Cookie: __cf_bm=rvy_ZQ1GQi0TAhZlDWMTRaTkygbe8gpYAQuTXzvbBAU-1760982803-1.0.1.1-ftZ7b61waV9NQYea5OQRXnldSvXzrDjHJZ_paRVcwhNhXnysNyymyRpY.cm7eL6q4Czn5R5aB0QMoxqagcAVcfQeGZn626xvo2E0JFk34XU; path=/; expires=Mon, 20-Oct-25 18:23:23 GMT; domain=.stackoverflow.com; HttpOnly
X-DNS-Prefetch-Control: off
Server: cloudflare
CF-RAY: 991a5fdabc4ce9a5-SCL

<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>cloudflare</center>
</body>
</html>



La respuesta que obtenemos del servidor web ubicado en el puerto 80 de `stackoverflow.com` es una **respuesta HTTP** (*HTTP Response*) que contiene a lo más 4096 *bytes*. La respuesta HTTP es una de 301 (moved permanently), donde se indica que para conectarse a StackOverflow hay que usar la nueva dirección indicada, usando HTTPS.

¿Qué significa esto? El puerto 80 es usado por los servidores. Se trata un servicio de **texto plano**, esto es, sin encriptación. La mayoría de los servidores web modernos utilizan una versión de HTTP con encriptación, conocida como HTTPS y que funciona en el puerto 443. Conectarse a un servidor web con HTTPS requiere operaciones de encriptación sobre el *socket* que complicarían innecesariamente este ejemplo.

#### Servidor TCP

> **NOTA**: Debido a la naturaleza de este contenido, esta sección contiene porciones de código separados que **no pueden ser ejecutados** con el fin de poder explicarlas de forma ordenada, ya que no es recomendable ejecutarlos en Jupyter Notebook. Si quieres ejecutarlo, en la carpeta `codigo/TCP` encontrarás un código similar al que veremos a continuación.

Un servidor debe escuchar las conexiones que le llegan a él en un puerto específico. Para ello, debemos "enlazar" o "asociar" un *socket* a ese puerto con el método `bind((host, port))`. El primer elemento de la tupla es el *hostname* del *host* en que estamos corriendo el servidor; el segundo elemento es el puerto donde queremos escuchar las conexiones.

Supongamos que queremos escuchar en el puerto 9000.

```python
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Poner como host 'localhost' indica que el host será nuestro computador
host = 'localhost'
port = 8998

sock.bind((host, port))
```

En el cliente no es necesario hacer un *bind* entre el *host* y el puerto, ya que el sistema operativo lo hace implícitamente a través del método `connect`, asignando al cliente un puerto aleatorio. Si el servidor al cual nos conectamos exigiera que el puerto de cada cliente esté en un rango específico, tendríamos que "enlazar" el cliente a un puerto específico también.

Ahora, empecemos a escuchar conexiones con el método `listen(conexiones_pendientes)`. El parámetro `conexiones_pendientes` indica cuantas conexiones **pendientes (sin aceptar)** pueden estar de forma paralela esperando a ser aceptadas por el servidor, antes de que este rechace cualquier conexión nueva sobre ese máximo.

```python
sock.listen(3) # Como máximo pueden haber hasta 3 clientes esperando ser aceptados al mismo tiempo.
```

Ahora, empecemos a aceptar conexiones. Para ello, ocupamos el método `accept()`, que nos retornará un _socket_ con el que podremos comunicarnos con ese cliente en específico, y la dirección de ese cliente (en específico, una tupla con su dirección IP y su puerto). El método esperará hasta que haya una conexión entrante; por lo tanto, si nadie se conecta, el programa quedará congelado.

```python
while True:
    socket_cliente, address = sock.accept()
    print("Conexión aceptada desde", address)
    socket_cliente.sendall("Gracias por conectarte\n".encode("utf-8"))
    socket_cliente.close()
```

Cuando queramos terminar el programa servidor, debemos cerrar el *socket* que estaba aceptando las conexiones.
```python
sock.close()
```

#### Ejemplo base

Hagamos el ejercicio completo, ejecuta los códigos y mira cómo se comportan. Puedes encontrar los códigos del servidor y el cliente en la carpeta `codigo/TCP`. Para ejecutarlos, debes abrir múltiples terminales, en una debes ejecutar el archivo del servidor y en las otras el archivo del cliente. 

Primero revisemos el código del servidor. Este se comporta igual que el revisamos anteriormente: recibe un cliente un cliente, le manda un mensaje y cierra la conexión. Adicionalmente, este servidor presenta una singularidad respecto al otro código, ya que el servidor cerrará después de aceptar cinco clientes.

```python
## Código servidor

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Poner como host 'localhost' indica que el host será nuestro computador
host = 'localhost'
port = 8998

sock.bind((host, port))
sock.listen()
print("[Servidor] Aceptando conexiones en el puerto", port)

counter = 0
while counter < 5:
    try:
        socket_cliente, address = sock.accept()
        print("[Servidor] Conexión aceptada desde", address)
        socket_cliente.sendall("Gracias por conectarte\n".encode("utf-8"))
        socket_cliente.close()
        counter += 1
    except ConnectionError:
        print("[Servidor] Ocurrió un error.")

print("[Servidor] Se han recibido 5 clientes, por lo que se cerrara la conexión")
sock.close()
```

En cuanto al cliente, este intentará conectarse al servidor y recibir el mensaje que envíe. Una vez recibido el mensaje, o en si sucede un error, el cliente cerrará la conexión con el servidor y se terminará el programa.
```python
## Código cliente

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Poner como host 'localhost' indica que el host será nuestro computador
host = 'localhost'
port = 8998

try:
    sock.connect((host, port))
    data = sock.recv(4096)
    print(data.decode("utf-8"))
except ConnectionError as e:
    print("[Cliente] Ocurrió un error")
finally:
    sock.close() 
```